In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import datetime
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from scipy.optimize import curve_fit
import glob
import os
import shutil
import plotly.graph_objects as go
from openpyxl import load_workbook

In [27]:
nox_path='data/T200_NOx_/2023-09-25_08.27.00_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_09_25_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_09_25_Channel1_Data.dat'
rh_path='data/Humidity_Sensors_/2023-09-25_08.27.01_PC14751_Humidity_Sensors.dat'

In [44]:
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

print(hono['Time_correct'][1])

2023-09-25 02:01:20.234000


In [29]:
dil_factor=1

df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']*dil_factor
df_no2_icad['no2']=no2['NO2 (ppb)']*dil_factor
df_nox_t200['no']=nox['NO']*dil_factor


In [30]:
average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index

df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index


In [36]:
df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
df

,hono,date_time,no2,no,dew_point
0,0.122500,2023-09-25 08:27:00,0.034567,0.595,0.000000
1,0.217500,2023-09-25 08:28:00,0.089150,0.920,7.050000
2,0.147333,2023-09-25 08:29:00,0.046700,1.170,17.251724
3,0.121167,2023-09-25 08:30:00,0.072350,1.430,16.686207
4,0.133200,2023-09-25 08:31:00,0.081867,1.685,16.165517
...,...,...,...,...,...
625,-0.014000,2023-09-25 18:52:00,0.001100,0.300,15.493103
626,-0.019333,2023-09-25 18:53:00,-0.010267,0.300,15.410345
627,0.471500,2023-09-25 18:54:00,0.005967,0.350,-4.448276
628,-0.202000,2023-09-25 18:55:00,0.004633,0.430,-21.793103


In [56]:
df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-25 16:53:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=506-10
df=df[time_start:time_start+60]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0925e NaNO3 (0.1M) + DHB (0.02M) 600 uL ',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               range=[-2,85],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=41,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)
# fig_1.add_vrect(x0=30, x1=36,annotation_text="UV-A lights off", annotation_position="top",annotation_font_size=14,fillcolor="grey", opacity=0.3,layer="below", line_width=0)
# fig_1.add_vrect(x0=36 , x1=36+10,annotation_text="bypass", annotation_position="top",annotation_font_size=14,fillcolor="white", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


         hono           date_time       no2    no  dew_point
506  0.005167 2023-09-25 16:53:00 -0.002217  0.53  17.213793


In [58]:
df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-09-25 18:03:00'])
# df.hono=df.hono-df.hono[46:76].mean()
time_start=576-10
df=df[time_start:time_start+30+30]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title='0925f NaNO3 (0.1M) + DHB (0.02M) 600 uL ',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', 
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=46,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
# fig_1.to_image('3.jpeg')


         hono           date_time     no2   no  dew_point
576  0.041833 2023-09-25 18:03:00  0.0091  0.2  17.055172
